In [89]:
from pinecone import Pinecone
import pinecone
import openai

pc= Pinecone(api_key="pcsk_65KdEB_3qYK83nZKETac9MTrVM2Sgtyb9Jw3HSRfG7FPJGZnbZ38TVTX1uXMYJTj2fTLk1")

index_name = "quickstart-notebook"

if not pc.has_index(name=index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model":"llama-text-embed-v2",
            "field_map":{"text": "chunk_text"}
        }
    )
index = pc.Index(index_name)


docs = [
    {"id": "1", "chunk_text": "Pinecone provides vector database as a service."},
    {"id": "2", "chunk_text": "Vector search is crucial for semantic similarity."},
    {"id": "3", "chunk_text": "Lexical search relies on keyword matching."}
]

index.upsert_records(
    namespace="example-namespace",
    records=docs
)

In [90]:
import time
time.sleep(10)

# View stats for the index
stats = index.describe_index_stats()
stats

{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'example-namespace': {'vector_count': 3}},
 'total_vector_count': 3,
 'vector_type': 'dense'}

In [94]:
def print_results(search_results):
    for hit in search_results['result']['hits']:
        print(f"id: {hit['_id']:<5} | score: {round(hit['_score'], 3):<5} | text: {hit['fields']['chunk_text']:<50}")


In [95]:
from pinecone import SearchQuery, SearchRerank

# Define the query
query = "Famous historical structures and monuments"

# Search the dense index
results = index.search(
    namespace="example-namespace",
    query=SearchQuery(
        top_k=10,
        inputs={'text': query}
    )
)

print_results(results)

id: 1     | score: 0.026 | text: Pinecone provides vector database as a service.   
id: 2     | score: -0.03 | text: Vector search is crucial for semantic similarity. 
id: 3     | score: -0.033 | text: Lexical search relies on keyword matching.        
